<a href="https://colab.research.google.com/github/jaycrossler/ai-training/blob/main/CrewAI%20Multiple%20Agents%20with%20search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain_community colab-xterm crewai 'crewai[tools]'
%load_ext colabxterm
# If asked to restart, accept and run this cell again

In [2]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [3]:
import subprocess
import time

In [4]:
server = subprocess.Popen(['ollama', 'serve'])
#time.sleep(60) # To make sure ollama is ready in subsequent cell if you are running all not cell at a time


In [5]:
llama3 = subprocess.Popen(['ollama', 'run', 'llama3'])

In [7]:
!ollama list

NAME             ID              SIZE      MODIFIED       
llama3:latest    365c0bd3c000    4.7 GB    22 seconds ago    


In [8]:
from langchain_community.llms import Ollama

# Initialize an instance of the Ollama model
llm = Ollama(model="llama3")

# Invoke the model to generate responses
response = llm.invoke("Tell me a joke")
print(response)

<ipython-input-8-41a9678af695>:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3")


Here's one:

Why couldn't the bicycle stand up by itself?

(Wait for it...)

Because it was two-tired!

Hope that made you laugh! Do you want to hear another?


In [9]:
from crewai.tools import BaseTool
class MyCustomDuckDuckGoTool(BaseTool):
    name: str = "DuckDuckGo Search Tool"
    description: str = "Search the web for a given query."

    def _run(self, query: str) -> str:
        duckduckgo_tool = DuckDuckGoSearchRun()

        response = duckduckgo_tool.invoke(query)

        return response

In [10]:
from crewai import Agent, Task, Crew, Process, LLM
from langchain_community.llms import Ollama
#llm = Ollama(model="llama3")

llm_o=LLM(model="ollama/llama3", base_url="http://localhost:11434")

researcher = Agent(
        role = "Researcher",
        goal = f"""To collaborate with your colleages to gather their views
        and opitions and get a final answer to complex questions""",
        verbose = True, # This way we can see the agents thoughts and messages
        #human_input=True, # Enable this if you want to be able to ask the user questions
        allow_delegation=True, # This allows to delegate tasks to other agents
        llm=llm_o,
        backstory = """You are an experienced researcher.
        You collaborate with expert colleages to get their views.
        You synthesize information in a simple and understandable way.
        """
    )


architecht = Agent(
        role = "System Architecht",
        goal = f"To collaborate with your colleages and provide your views as a system architecht",
        verbose = True, # This way we can see the agents thoughts and messages
        allow_delegation=False,
        llm=llm_o,
        backstory = """You are an experienced government software and system architecht.
        You can find answers to complex questions and work with colleagues.
        """
    )

commander = Agent(
        role = "commander",
        goal = f"To collaborate with your colleages and provide your views as a military leader and commander",
        verbose = True,# This way we can see the agents thoughts and messages
        allow_delegation=False,
        llm=llm_o,
        backstory = """You are an experienced Military Commander.
        You can find answers to complex questions and work with colleagues.
        """
    )

economist = Agent(
        role = "Economist",
        goal = f"To collaborate with your colleages and provide your views as an economist and acquisition expert",
        verbose = True,# This way we can see the agents thoughts and messages
        allow_delegation=False,
        llm=llm_o,
        backstory = """You are an experienced economis and government program acquisition expert.
        You can find answers to complex questions and work with colleagues.
        """
    )

LLM value is already an LLM object
LLM value is already an LLM object
LLM value is already an LLM object
LLM value is already an LLM object


In [11]:
task= Task(
    description="Describe a system architecture for a Government drone project that a Commander, Economist, and System Architecht can all agree upon?",
    expected_output="""
    The answer to the question: "Describe a system architecture for a Government drone project that a Commander, Economist, and System Architecht can all agree upon?"
    """,
    max_inter=5, #This defines the max amount of iterations an agent can perform to avoid infinite loops. Max=25
    tools=[MyCustomDuckDuckGoTool()],
    agent=researcher # We will delegate the task initially to the researcher
    )

In [ ]:
crew = Crew(
    agents=[researcher,architecht,commander,economist],
    model="ollama/llama3",
    tasks=[task],
    verbose=True, # This way we can see the agents thoughts and messages
    process=Process.sequential)

result = crew.kickoff()

# Agent: Researcher
## Task: Describe a system architecture for a Government drone project that a Commander, Economist, and System Architecht can all agree upon?
# Agent: commander
## Task: What are the primary considerations for designing a system architecture for a Government drone project?
